# Run a REopt API evaluation

## Initialization

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import copy
import os
from src.post_and_poll import get_api_results
API_KEY = 'DEMO_KEY'  # REPLACE WITH YOUR API KEY

In [2]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

In [3]:
"""
Here are some convenience definitions for using the Multi-scenario capabilities
"""
##############################################################################################################
inputs_path = os.path.join(".", 'inputs')
outputs_path = os.path.join(".", 'outputs')
loads_path = os.path.join(".", 'load_profiles')
rates_path = os.path.join(".", 'electric_rates')
##############################################################################################################

## Load a previously saved API response .json file instead of running REopt

## Scenario Inputs (POST), if wanting to do a new API call

In [6]:
post_1 = {
    "Site": {
        "longitude": -118.1164613,
        "latitude": 34.5794343
    },
    "PV": {
        "array_type": 0
    },
    "ElectricLoad": {
        "doe_reference_name": "RetailStore",
        "annual_kwh": 100000.0,
        "year": 2017
    },
    "ElectricTariff": {
        "urdb_label": "5ed6c1a15457a3367add15ae"
    },
    "Financial": {
        "elec_cost_escalation_rate_fraction": 0.026,
        "owner_discount_rate_fraction": 0.081,
        "analysis_years": 20,
        "offtaker_tax_rate_fraction": 0.4,
        "owner_tax_rate_fraction": 0.4,
        "om_cost_escalation_rate_fraction": 0.025
    }
}

### Load in a custom electric rate generated from https://reopt.nrel.gov/tool/custom_tariffs

In [7]:
load_electric_rate = "PGE_E20"
with open(os.path.join(rates_path, load_electric_rate + ".json"), 'r') as fp:
    rate_1 = json.load(fp)
post_1["ElectricTariff"]["urdb_response"] = rate_1

## Save the POST to a .json file for sharing or future loading in

In [9]:
# Convert python dictionary post into json and save to a .json file
post_save = post_1
post_name = "post_1"
with open(os.path.join(inputs_path, post_name + ".json"), 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the inputs/POST

In [10]:
# Load a json into a python dictionary
load_post = "post_1"
with open(os.path.join(inputs_path, load_post + ".json"), 'r') as fp:
    post_1 = json.load(fp)

## POST and poll (periodic GET request) the API to GET a new result, if not loading in a previous response. This may take a while!

### Note, the `api_url` in the `get_api_results` function below calls the **production server** hosted API (master/main branch/version, publicly accessible)

#### For calling a locally-hosted (localhost) API, see:
- https://github.com/NREL/REopt_API/wiki/3.-Calling-Locally-Hosted-API

#### For calling an API hosted on an NREL-internal server (only NREL users can access this), see:
- https://github.nrel.gov/REopt/API_scripts/wiki/API-URLs-for-NREL-internal-servers

`get_api_results` POST's your inputs to the API `job` endpoint, which provides a `run_uuid` if the input is valid, and then polls the `results` endpoint using the `run_uuid` until the results come back with a status other than `Optimizing...`.

`get_api_results` also saves the results (full API response, including inputs) to the `results_file`.

A log file is also created in the current working directory.

In [13]:
outputs_file_name = "results_file"
root_url = "https://developer.nrel.gov/api/reopt/dev"
api_response = get_api_results(post=post_1, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file=os.path.join(outputs_path, outputs_file_name + ".json"), 
                               run_id=None)

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/dev/job/?api_key=DEMO_KEY.
main         INFO     Polling https://developer.nrel.gov/api/reopt/dev/job/734c1363-21c8-466d-b384-7e91f7912056/results/?api_key=DEMO_KEY for results with interval of 5s...
main         INFO     Saved results to ./outputs/results_file.json


### If you get disconnected from the polling function but you think it ran, copy the run_uuid from the log above to manually GET the results:

In [19]:
api_response.keys()

dict_keys(['run_uuid', 'api_version', 'user_uuid', 'webtool_uuid', 'job_type', 'status', 'created', 'reopt_version', 'inputs', 'outputs', 'messages'])

In [20]:
run_uuid = api_response["run_uuid"]
results_url = root_url + '/job/' + run_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
api_response = json.loads(resp.text)

## Get summary of results

In [23]:
print("NPV ($) = ", api_response["outputs"]["Financial"]["npv"])
print("Capital Cost, Net ($) = ", api_response["outputs"]["Financial"]["lifecycle_capital_costs"])
tech_list = ["PV", "Wind", "Storage", "CHP", "Generator", "HotThermalStorage", "ColdThermalStorage", "AbsorptionChiller", "GHP", "Boiler", "SteamTurbine"]
for tech in tech_list:
    if tech in post_1.keys():
        if tech == "GHP":
            print("GHX Number of Boreholes = ", api_response["outputs"][tech]["ghpghx_chosen_outputs"].get("number_of_boreholes"))
            print("GHP Heat Pump Capacity (ton) = ", api_response["outputs"][tech]["ghpghx_chosen_outputs"].get("peak_combined_heatpump_thermal_ton"))
        for size_name_value in [(key, val) for key, val in api_response["outputs"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])               

NPV ($) =  48212.04
Capital Cost, Net ($) =  23567.331
PV size_kw  =  37.0196


### Here are some results keys examples:

In [24]:
api_response.keys()

dict_keys(['run_uuid', 'api_version', 'user_uuid', 'webtool_uuid', 'job_type', 'status', 'created', 'reopt_version', 'inputs', 'outputs', 'messages'])

In [25]:
api_response["status"]

'optimal'

In [26]:
api_response["messages"]

{'info': 'When using doe_reference_name or blended_doe_reference_names for ElectricLoad the year is set to 2017 because the DoE load profiles start on a Sunday.',
 'errors': {},
 'warnings': {},
 'has_stacktrace': False}

In [27]:
for k in api_response["outputs"].keys():
    print(k)

Financial
ElectricTariff
ElectricUtility
ElectricLoad
Site
PV


## Save API response into a JSON file for later use

In [28]:
response_save = api_response
file_name_to_save = "response_1"
with open(os.path.join(outputs_path, file_name_to_save + ".json"), 'w') as fp:
    json.dump(response_save, fp)